In [ ]:
import requests
import psycopg2
from datetime import datetime
import time
import urllib3

# Disable SSL warnings for demonstration only (not recommended for production)
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Database connection — update with your AWS RDS credentials
conn = psycopg2.connect(
    dbname="riderts",
    user="postgres",           # <-- your DB username
    password="Livetolove921",  # <-- your DB password
    host="your-rds-endpoint",  # <-- your RDS endpoint
    port="5432"
)
cur = conn.cursor()

# Public API endpoints for real-time vehicle locations
urls = [
    "https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=1,2,3,5,6,7,8,9,10,11&tmres=s&format=json",
    "https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=12,13,15,16,17,20,23,24,25,26&tmres=s&format=json",
    "https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=27,33,34,35,37,38,43,46,75,119&tmres=s&format=json",
    "https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=122,121,76,78,125,126,127,150,711&tmres=s&format=json",
    "https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=11,1,12,27,122,118,76,78&tmres=s&format=json",
    "https://riderts.app/bustime/api/v3/getvehicles?key=RrQJYQvn2PpvPCRUVQepQvMnc&rt=11,21,28,36,40,121&tmres=s&format=json"
]

def fetch_vehicles():
    """Fetch vehicle data from all endpoints."""
    all_data = []
    for url in urls:
        try:
            response = requests.get(url, verify=False, timeout=10)
            data = response.json().get('bustime-response', {}).get('vehicle', [])
            all_data.extend(data)
        except Exception as e:
            print(f"[{datetime.utcnow()}] Error fetching {url}: {e}")
    return all_data

# Create table if it doesn't exist
cur.execute("""
    CREATE TABLE IF NOT EXISTS vehicles (
        timestamp     TIMESTAMPTZ,
        vehicle_id    TEXT,
        rt            TEXT,
        lat           DOUBLE PRECISION,
        lon           DOUBLE PRECISION,
        hdg           INTEGER,
        pdist         DOUBLE PRECISION,
        des           TEXT,
        spd           DOUBLE PRECISION,
        psgld         TEXT
    )
""")
conn.commit()

print("Database table ready. Starting data collection...")

try:
    while True:
        vehicles = fetch_vehicles()
        now = datetime.utcnow()

        for v in vehicles:
            try:
                cur.execute("""
                    INSERT INTO vehicles (
                        timestamp, vehicle_id, rt, lat, lon, hdg, pdist, des, spd, psgld
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                    now,
                    v.get('vid'),
                    v.get('rt'),
                    float(v.get('lat', 0)),
                    float(v.get('lon', 0)),
                    int(v['hdg']) if v.get('hdg') else None,
                    float(v.get('pdist', 0)),
                    v.get('des'),
                    float(v['spd']) if v.get('spd') else None,
                    v.get('psgld')
                ))
            except Exception as e:
                print(f"[{now}] Insert error for vehicle {v.get('vid')}: {e}")

        conn.commit()
        print(f"[{now}] Inserted {len(vehicles)} records.")
        time.sleep(15)

except KeyboardInterrupt:
    print("Stopped by user.")
finally:
    cur.close()
    conn.close()
